# Train SALLY ensemble

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'

## Settings

In [5]:
n_estimators = 10

## Training function

In [6]:
def train_ensemble(filename, use_tight_cuts=True, n_estimators=n_estimators, **kwargs):
    cut_label = '_tight' if use_tight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators, debug=True)

    ensemble.train_all(
        method='sally',
        x_filename=[sample_dir + 'train_local{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_local{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.save(model_dir + 'sally_ensemble_' + filename)

## All observables

In [ ]:
train_ensemble(
    'all',
    use_tight_cuts=False,
    validation_split=0.5,
    early_stopping=True
)

15:16 madminer.ml          INFO    Training 10 estimators in ensemble
15:16 madminer.ml          INFO    Training estimator 1 / 10 in ensemble
15:16 madminer.ml          INFO    Starting training
15:16 madminer.ml          INFO      Method:                 sally
15:16 madminer.ml          INFO      Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/x_train_0.npy
15:16 madminer.ml          INFO                     t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_local/t_xz_train_0.npy
15:16 madminer.ml          INFO      Features:               all
15:16 madminer.ml          INFO      Method:                 sally
15:16 madminer.ml          INFO      Hidden layers:          (100, 100)
15:16 madminer.ml          INFO      Activation function:    tanh
15:16 madminer.ml          INFO      Batch size:             128
15:16 madminer.ml          INFO      Trainer:         

In [ ]:
train_ensemble(
    'all_tight',
    use_tight_cuts=True,
    validation_split=0.5,
    early_stopping=True
)

## Minimal observable basis (no jets)

In [ ]:
min_obs = [0,1] + list(range(4,12)) + list(range(16,33))

In [ ]:
train_ensemble(
    'minimal',
    use_tight_cuts=False,
    features=[min_obs for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True
)

In [ ]:
train_ensemble(
    'minimal_tight',
    use_tight_cuts=True,
    features=[min_obs for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True
)

## Just resurrection phi

In [ ]:
train_ensemble(
    'phi_tight',
    use_tight_cuts=True,
    features=[[32] for _ in range(n_estimators)],
    validation_split=0.5,
    early_stopping=True,
)